In [6]:
import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.sparse.linalg import eigsh
import pickle
from copy import deepcopy
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity
import time

from oft import oft
from tools.classical import find_ideal_heisenberg, trotter_step_heisenberg_qt, ham_evol_qt, hamiltonian_matrix

np.random.seed(667)
# 3-6, 4-8, 5-9, 6-12, 7-14, 8-16 (6 mins)
num_qubits = 6
num_energy_bits = 12
delta = 0.01
eps = 0.1
sigma = 5
bohr_bound = 2 ** (-num_energy_bits + 1)
beta = 1
eig_index = 2
mix_time = 4

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=True)
eigenstate_with_scipy = eigsh(hamiltonian.qt.full(), k=1, sigma=hamiltonian.spectrum[eig_index])[1].flatten()
initial_state = eigenstate_with_scipy / np.linalg.norm(eigenstate_with_scipy)

initial_dm = DensityMatrix(initial_state).data

N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
time_labels = N_labels
phase = 17

site_list = [qt.qeye(2) for _ in range(num_qubits)]
x_jump_ops = []
for q in range(num_qubits):
    site_list[q] = qt.sigmax()
    x_jump_ops.append(qt.tensor(site_list).full())

rand_jump_index = np.random.randint(0, len(x_jump_ops))
jump_op = x_jump_ops[rand_jump_index]

gauss = lambda t: np.exp(-t**2 / (4 * sigma**2))
gauss_values = gauss(time_labels)
normalized_gauss_values_bin_order = (gauss_values / np.sqrt(np.sum(gauss_values**2)))

# if hamiltonian is None and trotter is not None:  #* Trotterized
#     time_evolution = lambda n: np.linalg.matrix_power(trotter, n)
# elif hamiltonian is not None and trotter is None:  #* Exact
# time_evolution_fn = lambda t: expm(1j * t * hamiltonian.qt.full())


diag_ham = np.diagflat(hamiltonian.spectrum)
time_evolution_fn = lambda t: expm(1j * t * diag_ham) #! Wrong it needs the U, U^dag with eigenstates, so we do need H ES after all #FIXME:
#* 1) Prediagonalizing and using that instead of this fn, with expm in the loop
#* 2) Preexponentiating the eigenenergies, einsum
time_evolutions_fn = lambda times: np.stack([time_evolution_fn(t) for t in times], axis=0)


Original spectrum:  [-5.3733e+00 -4.6135e+00 -4.2413e+00 -3.2941e+00 -3.0120e+00 -2.9785e+00
 -2.8830e+00 -2.4277e+00 -2.3907e+00 -2.3581e+00 -2.1773e+00 -2.1104e+00
 -2.0977e+00 -2.0808e+00 -2.0593e+00 -1.7782e+00 -1.6481e+00 -1.4854e+00
 -1.3350e+00 -1.1192e+00 -1.1023e+00 -9.8890e-01 -8.5270e-01 -8.3080e-01
 -7.0020e-01 -6.8570e-01 -6.0500e-01 -3.0880e-01 -2.5040e-01 -2.1570e-01
  6.0000e-04  1.2910e-01  1.6440e-01  4.0150e-01  4.1600e-01  4.9720e-01
  5.6900e-01  1.0180e+00  1.0341e+00  1.0678e+00  1.0958e+00  1.1785e+00
  1.3436e+00  1.3746e+00  1.3944e+00  1.5125e+00  1.5396e+00  1.6083e+00
  1.7168e+00  1.7549e+00  1.7862e+00  1.8819e+00  2.0236e+00  2.0415e+00
  2.2986e+00  2.3863e+00  2.4917e+00  2.5866e+00  2.7019e+00  2.8579e+00
  2.9588e+00  3.1381e+00  3.8554e+00  5.1788e+00]
Ideal spectrum:  [-0.      0.0324  0.0483  0.0887  0.1007  0.1021  0.1062  0.1256  0.1272
  0.1286  0.1363  0.1391  0.1397  0.1404  0.1413  0.1533  0.1589  0.1658
  0.1722  0.1814  0.1821  0.187   0.1

In [7]:
t0 = time.time()
oft_op = np.zeros_like(jump_op, dtype=np.complex128)

for i, t in enumerate(time_labels):
    oft_op += (np.exp(-1j * 2 * np.pi * phase * t / N)  #!
                * normalized_gauss_values_bin_order[i] 
                * time_evolution_fn(t) @ jump_op @ time_evolution_fn(-t))
    
oft_op /= np.sqrt(N)
print(f'Time to build the OFT operator: {time.time() - t0:.2f} s')

Time to build the OFT operator: 1.64 s


In [20]:
time_evolutions_plus = np.stack([time_evolution_fn(t) for t in time_labels], axis=0)
time_evolutions_minus = time_evolutions_plus.conj()

In [17]:
phase_factors = np.exp(- 1j * 2 * np.pi * phase * time_labels / N)
t1 = time.time()
# print(jump_op.shape)
oft_op_np = np.einsum('i, i, ijk, km, imn -> jn', 
                      normalized_gauss_values_bin_order, phase_factors, time_evolutions_plus, jump_op, time_evolutions_minus, 
                      optimize=True) / np.sqrt(N)

print(f'Time to build the OFT operator with einsum: {time.time() - t1:.2f} s')

Time to build the OFT operator with einsum: 4.20 s


In [10]:

# print(oft_op_np.shape)
np.isclose(oft_op, oft_op_np, atol=1e-7).all()

True